## Imports

In [36]:
import re
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

## Utils

In [37]:
def parse_performance_data(text):
    pattern = re.compile(
        r"Algorithm:\s*(?P<Algorithm>\S+)\s*"
        r"Instance:\s*(?P<Instance>\S+)\s*"
        r"Initial Score:\s*(?P<Initial_Score>\d+)\s*"
        r"Score:\s*(?P<Score>\d+)\s*"
        r"Number\s*of\s*Evaluations:\s*(?P<Number_of_Evaluations>\d+)\s*"
        r"Number\s*of\s*Performed\s*Moves:\s*(?P<Number_of_Performed_Moves>\d+)\s*"
        r"Number\s*of\s*Best\s*Solution\s*Updates:\s*(?P<Number_of_Best_Solution_Updates>\d+)\s*"
        r"Solution:\s*(?P<Solution>[\d\s]+)\s*"
        r"Optimal\s*Score:\s*(?P<Optimal_Score>\d+)\s*"
        r"Optimal\s*Solution:\s*(?P<Optimal_Solution>[\d\s]+)"
    )
    
    rows = []
    for match in pattern.finditer(text):
        row = match.groupdict()
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    numerical_cols = ['Initial_Score', 'Score', 'Number_of_Evaluations', 'Number_of_Performed_Moves', 'Number_of_Best_Solution_Updates', 'Optimal_Score']
    df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric)
    
    return df

def parse_runtime_data(text):
    pattern = re.compile(
        r"Algorithm:\s*(?P<Algorithm>\S+)\s*"
        r"Instance:\s*(?P<Instance>\S+)\s*"
        r"Runtime:\s*(?P<Runtime>\S+)\s*"
    )
    
    rows = []
    for match in pattern.finditer(text):
        row = match.groupdict()
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    numerical_cols = ['Runtime']
    df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric)
    
    return df

## Read Data

In [38]:
with open('performance_results.txt', 'r') as file:
    data = file.read()

df_performance = parse_performance_data(data)

In [39]:
df_performance.tail()

,Algorithm,Instance,Initial_Score,Score,Number_of_Evaluations,Number_of_Performed_Moves,Number_of_Best_Solution_Updates,Solution,Optimal_Score,Optimal_Solution
4795,rsteepestLS,bur26h,7827571,7100009,8775,26,26,21 15 13 2 5 7 24 14 0 20 19 3 17 6 25 9 18 8 ...,7098658,11 2 13 12 6 10 25 8 1 21 4 20 7 18 14 15 9 19...
4796,rsteepestLS,bur26h,7881818,7142497,8775,26,26,5 16 14 25 20 10 2 11 12 13 6 4 8 18 7 22 3 17...,7098658,11 2 13 12 6 10 25 8 1 21 4 20 7 18 14 15 9 19...
4797,rsteepestLS,bur26h,7938698,7121974,5850,17,17,25 13 22 10 5 16 23 0 14 20 18 17 19 3 1 2 6 4...,7098658,11 2 13 12 6 10 25 8 1 21 4 20 7 18 14 15 9 19...
4798,rsteepestLS,bur26h,8092892,7147232,5850,17,17,5 25 24 16 23 22 21 13 10 2 6 18 4 8 0 11 19 1...,7098658,11 2 13 12 6 10 25 8 1 21 4 20 7 18 14 15 9 19...
4799,rsteepestLS,bur26h,7609082,7146811,8450,25,25,16 5 14 7 6 10 2 12 11 13 4 0 8 18 1 25 3 19 1...,7098658,11 2 13 12 6 10 25 8 1 21 4 20 7 18 14 15 9 19...


In [40]:
with open('runtime_results.txt', 'r') as file:
    data = file.read()

df_runtime = parse_runtime_data(data)

In [41]:
df_runtime.head()

,Algorithm,Instance,Runtime
0,heuristic,bur26a,6890.0
1,random,bur26a,858.0
2,antiheuristic,bur26a,8290.0
3,greedyLS,bur26a,4191900.0
4,steepestLS,bur26a,2520280.0


In [42]:
df_runtime[df_runtime.Algorithm == "greedyLS"].Runtime.mean()

3410995.0

## Analysis

### 8 Selected instances:

R.E. Burkard and J. Offermann [BuOf:77]
The data of the first matrix correspond to the typing-time of an average stenotypist, while the second matrix describes the frequency of pairs of letters in different languages taken over 100,000 pairs for examples a-f and over 187,778 pairs for examples g-h. (Note that the solutions are not scaled for a flow matrix of 100,000 pairs anymore.) One also distinguishes between two types of typewriter keyboards. The instances are asymmetric.

Selected due to the fact that all instances are of the same size.<br>
***Bur26a, Bur26b, Bur26c, Bur26d, Bur26e, Bur26f, Bur26g, Bur26h***

### Neighborhood used: pair swap
### Neighborhood size: n(n-1)/2 ? 

### Comparison of the performance of 5 algorithms and implemented types of neighborhoods on all problem instances – plots

- Running time (average)
- Quality = distance from the optimum (according to what measure?), the average and the best case (optionally: also the worst case).
- Efficiency of algorithms (average) – i.e., quality over time (suggest a good measure and justify your choice)
- G,S: average number of algorithm steps (step = changing the current solution)
- G,S,RS,RW: average number of evaluated (i.e., visited – full or partial evaluation) solutions

For the averages, we assess the stability of the results (standard deviations should always be shown along with the averages).

In [43]:
df_performance["Solution_Quality"] = (df_performance["Score"] - df_performance["Optimal_Score"]) / df_performance["Optimal_Score"]